In [63]:
import csv
import requests
from bs4 import BeautifulSoup as bs
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from sklearn.discriminant_analysis import StandardScaler
from requests.adapters import HTTPAdapter, Retry
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer


def find_divtag(url, input_text):
    field_label = []
    html_tag = []
    tag_attrs = []
    text_content = []
    try: 
        # connect to URL
        # define default user agent to prevent status code 403
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
        #response = requests.get(url, headers = headers)
        s = requests.Session()
        retries = Retry(total=5,
                        backoff_factor=0.1,
                        status_forcelist=[ 500, 502, 503, 504 ])
        s.mount('http://', HTTPAdapter(max_retries=retries))
        response = s.get(url, headers=headers, timeout=3)

        # http request success
        if response.status_code == 200:
            soup = bs(response.content, "html.parser")
            # get divtag for every field
            for text in input_text:
                # return smallest html element that contains text (div/span/etc)
                result = [tag for tag in soup.find_all() if text in str(tag)]
                if result:
                    divtag = str(min(result, key=lambda element: len(str(element.get_text(strip=True))))) 
                    if divtag != "":           
                        field_label.append(input_text.index(text)) # 0 = name; 1 = address; 2 = zip; 3 = phone num
                        html_tag.append(divtag)
                        text_content.append(text)

                        tag_list = bs(divtag, 'html.parser').find().attrs
                        if tag_list is None:
                            tag_attrs.append([])
                        else:
                            tag_attrs.append(tag_list)
            return field_label, html_tag, tag_attrs, text_content
        # http request failed
        else:
            print(f"Status code: {response.status_code}. Failed to retrieve content from {url}.")
            return None
    # unknown exception (usually about network connectivity)
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
    
def clean_csv():
    # read in largest scraping dataset
    df = pd.read_csv('resources_large.csv', sep=";")
    columns_to_keep = ['resource_id', 'name', 'address_1', "zip", "Phone Number", "Website where info found"]  # Replace with your actual column names

    # create a new df with only the essential fields
    df_filtered = df[columns_to_keep]
    df_cleaned = df_filtered.rename(columns={"Website where info found": "url", "address_1": "address"}).dropna(subset=["url"])
    print("Sample dataset size: " + str(df_cleaned.shape))

    # option to export small subset temporarily only for testing
    #df_small = df_cleaned.head(1000)
    df_cleaned.to_csv('resources_cleaned.csv', sep=";", index=False)

def train_svm_model(x, y):
    # train/test split
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state = 101) 
    model = svm.SVC() 
    model.fit(X_train, y_train) 
    
    # performance before tuning
    predictions = model.predict(X_test) 
    print(classification_report(y_test, predictions)) 

    # hyperperameter tuning
    param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
    grid = GridSearchCV(svm.SVC(), param_grid, cv = 5, refit = True, verbose = 3) 
    grid.fit(X_train, y_train) 
    print(grid.best_params_) 
    print(grid.best_estimator_)     
    
    # final performance
    grid_predictions = grid.predict(X_test) 
    print(classification_report(y_test, grid_predictions)) 
   

In [64]:
# get rid of rows with empty urls, rename fields, etc
clean_csv()

# extract all raw features
data = {}
file_input = input("Enter a CSV containing clinic data: ")
with open(file_input, mode='r') as file:
    reader = csv.reader(file, delimiter=";")
    # print field types
    header = next(reader, None)
    if header:
        print(f"Fields: {header}") 
    # exclude last index since it's URL
    field_len = len(header) - 1 

    # extract labeled input for SVM
    field_label = []    # supervised data
    html_tag = []   # feature for raw html tags
    tag_attrs = []  # feature for tag attributes
    text_content = [] # feature for text CBOW
    for row in reader:
        print("Extracting features for clinic id " + row[0])
        res = find_divtag(row[header.index('url')], row[1:-1])
        if res is not None:
            field_label_temp, html_tag_temp, tag_attrs_temp, text_content_temp = res        
            field_label += field_label_temp
            html_tag += html_tag_temp
            tag_attrs += tag_attrs_temp
            text_content += text_content_temp

Sample dataset size: (4475, 6)
Fields: ['resource_id', 'name', 'address', 'zip', 'Phone Number', 'url']
Extracting features for clinic id 3
Extracting features for clinic id 13
Extracting features for clinic id 14
Extracting features for clinic id 16
Extracting features for clinic id 17
Extracting features for clinic id 18
Extracting features for clinic id 20
Extracting features for clinic id 21
Extracting features for clinic id 23
Extracting features for clinic id 24
Extracting features for clinic id 25
Extracting features for clinic id 26
Extracting features for clinic id 27
Extracting features for clinic id 28
Extracting features for clinic id 37
Extracting features for clinic id 40
Extracting features for clinic id 48
Extracting features for clinic id 49
Extracting features for clinic id 50
Extracting features for clinic id 51
Status code: 404. Failed to retrieve content from https://snowlinehospice.org/our-services/hospice-services.
Extracting features for clinic id 52
Extracting 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Extracting features for clinic id 391
Extracting features for clinic id 398
Extracting features for clinic id 400
Error: HTTPSConnectionPool(host='unininc.org', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x175a8bf50>: Failed to resolve 'unininc.org' ([Errno 8] nodename nor servname provided, or not known)"))
Extracting features for clinic id 401
Extracting features for clinic id 403
Extracting features for clinic id 404
Extracting features for clinic id 414
Extracting features for clinic id 415
Error: HTTPSConnectionPool(host='wellpathcarecenters.com', port=443): Max retries exceeded with url: /our-services/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x2ea5293d0>, 'Connection to wellpathcarecenters.com timed out. (connect timeout=3)'))
Extracting features for clinic id 417
Extracting features for clinic id 419
Extracting features for clinic id 420
Extracting features for cli

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Extracting features for clinic id 547
Extracting features for clinic id 552
Extracting features for clinic id 553
Error: HTTPSConnectionPool(host='www.latterdaysaintjoblue%20shield.org', port=443): Max retries exceeded with url: /ers/ct/?lang=eng (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x2dcd24b90>: Failed to resolve 'www.latterdaysaintjoblue%20shield.org' ([Errno 8] nodename nor servname provided, or not known)"))
Extracting features for clinic id 557
Extracting features for clinic id 559
Extracting features for clinic id 560
Extracting features for clinic id 563
Error: HTTPSConnectionPool(host='horizonpersonnelservices.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x2dc947c50>: Failed to resolve 'horizonpersonnelservices.com' ([Errno 8] nodename nor servname provided, or not known)"))
Extracting features for clinic id 565
Extracting features for clinic id 566
Extrac

...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Extracting features for clinic id 10853
Extracting features for clinic id 10854
Extracting features for clinic id 10855
Extracting features for clinic id 10857
Extracting features for clinic id 10858
Extracting features for clinic id 10859
Extracting features for clinic id 10860
Extracting features for clinic id 10861
Extracting features for clinic id 10862
Extracting features for clinic id 10863
Extracting features for clinic id 10864
Extracting features for clinic id 10865
Extracting features for clinic id 10866
Extracting features for clinic id 10867
Extracting features for clinic id 10868
Extracting features for clinic id 10869
Extracting features for clinic id 10870
Extracting features for clinic id 10871
Extracting features for clinic id 10872
Extracting features for clinic id 10873
Extracting features for clinic id 10874
Extracting features for clinic id 10876
Extracting features for clinic id 10877
Extracting features for clinic id 10878
Extracting features for clinic id 10879


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Extracting features for clinic id 10934
Extracting features for clinic id 10935
Extracting features for clinic id 10936
Extracting features for clinic id 10937
Extracting features for clinic id 10938
Extracting features for clinic id 10939
Extracting features for clinic id 10940
Extracting features for clinic id 10941
Extracting features for clinic id 10942
Extracting features for clinic id 10943
Error: HTTPSConnectionPool(host='www.foothillpainmanagement.com', port=443): Max retries exceeded with url: /telehealth (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate (_ssl.c:1006)')))
Extracting features for clinic id 10944
Extracting features for clinic id 10945
Extracting features for clinic id 10946
Extracting features for clinic id 10947
Error: HTTPSConnectionPool(host='ossglendale.com', port=443): Max retries exceeded with url: /telehealth/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnect

KeyboardInterrupt: 

In [65]:
# check the labels
print("Number of samples: " + str(len(field_label)))

Number of samples: 7478


In [66]:
# convert features to vectors that can be used in SVM

# numeric representation of the raw html tags
vectorizer = CountVectorizer()
html_features = vectorizer.fit_transform(html_tag).toarray()

# extract relevant attribute features that's indicative of field classification
df = pd.DataFrame(tag_attrs)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# sort by most avail entries and choose the first 5 cols
nan_counts = df.isna().sum()
count_sort_col = nan_counts.sort_values().index
#print(df_ordered["name"])

In [67]:
df_ordered = df[count_sort_col].astype(str)

In [68]:
mlb = MultiLabelBinarizer()

# create one hot encoding of attribute "class"
mask = df_ordered['class'].notnull()
arr = mlb.fit_transform(df_ordered.loc[mask, 'class'].dropna().str.strip('[]').str.split(','))
class_labels = (pd.DataFrame(arr, index=df_ordered.index[mask], columns=mlb.classes_)
               .reindex(df_ordered.index, fill_value=0))
class_labels = class_labels.drop('nan', axis=1)

#print(class_labels)


In [69]:
# create one hot encoding of attribute "name"

df_ordered["type"] = df_ordered["type"].fillna("nan")
df_ordered["type"] = df_ordered["type"].values.reshape(-1, 1).tolist()
name_labels = pd.DataFrame(mlb.fit_transform(df_ordered['type']),columns=mlb.classes_, index=df_ordered.index).drop("nan", axis = 1)

print(html_features.shape)
print(class_labels.shape)
print(name_labels.shape)

(7478, 124511)
(7478, 1698)
(7478, 15)


In [70]:
# create features from text content
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the text data
text_representation = vectorizer.fit_transform(text_content)
text_features = text_representation.toarray()
print(text_features.shape)


(7478, 4218)


In [71]:
# reduce dimension of html_features
pca = PCA(n_components=0.95)
html_features_reduced = pca.fit_transform(html_features)

# add all three feature matrices
feature_matrix = []
for i in range(len(html_features)):
    feature_matrix.append(html_features_reduced[i].tolist() + class_labels.iloc[i].tolist() + name_labels.iloc[i].tolist() + text_features[i].tolist())
    
# Standardize the feature matrix (seems to make accuracy worse in this case, prob since the features aren't linearly correlated)
# feature_matrix = np.array(feature_matrix)
# feature_matrix_scaled = StandardScaler().fit_transform(feature_matrix)

In [ ]:
print(html_features_reduced.shape)

(861, 4)
[[-137.35467461  -60.86861489  -19.99540301  -21.05451257]
 [-135.81921771  -57.55500521   -5.33546479  -21.2092331 ]
 [-137.0775953   -60.81590577  -19.92451989  -20.7604837 ]
 ...
 [-137.33053281  -60.81907815  -19.94491252  -21.02805763]
 [-136.54198078  -60.63692226  -19.72411718  -20.084282  ]
 [-136.54198078  -60.63692226  -19.72411718  -20.084282  ]]


In [72]:
# train/test support vector machine
train_svm_model(feature_matrix, field_label)

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.48      0.18      0.26       469
           1       0.00      0.00      0.00       611
           2       0.32      0.94      0.48       716
           3       0.00      0.00      0.00       448

    accuracy                           0.34      2244
   macro avg       0.20      0.28      0.19      2244
weighted avg       0.20      0.34      0.21      2244

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.463 total time= 1.4min
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.482 total time= 1.4min
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.486 total time= 1.5min
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.497 total time= 1.4min
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.489 total time= 1.4min
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.420 total time= 1.5min
[CV 2/5] END ....